In [1]:
import pandas as pd

# 파일 경로
mapping_path = "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/data/yolo_mapping.csv"

# CSV 로드
df = pd.read_csv(mapping_path, encoding="utf-8-sig")

# class_id, categories_name 컬럼만 사용
if not {"class_id", "categories_name"}.issubset(df.columns):
    raise ValueError("❌ yolo_mapping.csv에 'class_id' 또는 'categories_name' 컬럼이 없습니다.")

# 정렬 (class_id 기준)
df = df.sort_values(by="class_id").reset_index(drop=True)

# YAML 형식으로 출력
print("names:")
for _, row in df.iterrows():
    class_id = int(row["class_id"])
    name = str(row["categories_name"]).strip().replace('"', "'")  # 큰따옴표 이스케이프
    print(f"  {class_id}: \"{name}\"")

names:
  0: "보령부스파정 5mg"
  1: "뮤테란캡슐 100mg"
  2: "일양하이트린정 2mg"
  3: "기넥신에프정(은행엽엑스)(수출용)"
  4: "무코스타정(레바미피드)(비매품)"
  5: "알드린정"
  6: "뉴로메드정(옥시라세탐)"
  7: "타이레놀정500mg"
  8: "에어탈정(아세클로페낙)"
  9: "삼남건조수산화알루미늄겔정"
  10: "타이레놀이알서방정(아세트아미노펜)(수출용)"
  11: "삐콤씨에프정 618.6mg/병"
  12: "조인스정 200mg"
  13: "쎄로켈정 100mg"
  14: "리렉스펜정 300mg/PTP"
  15: "아빌리파이정 10mg"
  16: "자이프렉사정 2.5mg"
  17: "다보타민큐정 10mg/병"
  18: "써스펜8시간이알서방정 650mg"
  19: "에빅사정(메만틴염산염)(비매품)"
  20: "리피토정 20mg"
  21: "크레스토정 20mg"
  22: "가바토파정 100mg"
  23: "동아가바펜틴정 800mg"
  24: "오마코연질캡슐(오메가-3-산에틸에스테르90)"
  25: "란스톤엘에프디티정 30mg"
  26: "리리카캡슐 150mg"
  27: "종근당글리아티린연질캡슐(콜린알포세레이트)"
  28: "콜리네이트연질캡슐 400mg"
  29: "트루비타정 60mg/병"
  30: "스토가정 10mg"
  31: "노바스크정 5mg"
  32: "마도파정"
  33: "플라빅스정 75mg"
  34: "엑스포지정 5/160mg"
  35: "펠루비정(펠루비프로펜)"
  36: "아토르바정 10mg"
  37: "라비에트정 20mg"
  38: "리피로우정 20mg"
  39: "자누비아정 50mg"
  40: "맥시부펜이알정 300mg"
  41: "메가파워정 90mg/병"
  42: "쿠에타핀정 25mg"
  43: "비타비백정 100mg/병"
  44: "놀텍정 10mg"
  45: "자누메트정 50/850mg"
  46: "큐시드정 31.5mg/

In [2]:
import os
import pandas as pd

# ------------------------------------------------------
# 1️⃣ 경로 설정
# ------------------------------------------------------
annotations_csv = "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/data/project_annotations.csv"
mapping_csv     = "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/data/yolo_mapping.csv"
output_dir      = "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/data/yolo_label_integration"

# ------------------------------------------------------
# 2️⃣ 폴더 준비
# ------------------------------------------------------
os.makedirs(output_dir, exist_ok=True)

# ------------------------------------------------------
# 3️⃣ 데이터 로드
# ------------------------------------------------------
ann_df = pd.read_csv(annotations_csv)
map_df = pd.read_csv(mapping_csv)

# 데이터 전처리
ann_df["dl_idx"] = ann_df["dl_idx"].astype(str).str.strip()
map_df["categories_id"] = map_df["categories_id"].astype(str).str.strip()

# ------------------------------------------------------
# 4️⃣ dl_idx → class_id 매핑
# ------------------------------------------------------
merged = ann_df.merge(
    map_df[["categories_id", "class_id"]],
    left_on="dl_idx",
    right_on="categories_id",
    how="inner"
)

# ------------------------------------------------------
# 5️⃣ YOLO 포맷 변환 함수
# ------------------------------------------------------
def bbox_to_yolo(bbox, img_w, img_h):
    try:
        x, y, w, h = [float(v) for v in bbox.strip("[]").split(",")]
    except Exception:
        return None  # 잘못된 bbox는 건너뜀

    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    w_norm = w / img_w
    h_norm = h / img_h
    return x_center, y_center, w_norm, h_norm

# ------------------------------------------------------
# 6️⃣ 파일 단위로 YOLO txt 생성
# ------------------------------------------------------
for file_name, group in merged.groupby("file_name"):
    label_lines = []

    for _, row in group.iterrows():
        bbox_str = str(row["bbox"])
        result = bbox_to_yolo(bbox_str, row["width"], row["height"])
        if result is None:
            continue

        x_center, y_center, w_norm, h_norm = result
        class_id = int(row["class_id"])
        label_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}")

    # 라벨 파일 저장
    label_path = os.path.join(output_dir, os.path.splitext(file_name)[0] + ".txt")
    with open(label_path, "w", encoding="utf-8") as f:
        f.write("\n".join(label_lines))

print(f"✅ YOLO 포맷 라벨 파일 생성 완료 → {output_dir}")
print(f"총 생성된 파일 수: {len(merged['file_name'].unique())}")

✅ YOLO 포맷 라벨 파일 생성 완료 → /Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/data/yolo_label_integration
총 생성된 파일 수: 1489


In [5]:
import os
import pandas as pd

# ------------------------------------------------------
# 1️⃣ 경로 설정
# ------------------------------------------------------
annotations_csv = "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/data/augmented_dlidx_samples.csv"
mapping_csv     = "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/data/yolo_mapping.csv"
output_dir      = "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/data/yolo_label_integration"

# ------------------------------------------------------
# 2️⃣ 출력 폴더 준비
# ------------------------------------------------------
os.makedirs(output_dir, exist_ok=True)

# ------------------------------------------------------
# 3️⃣ 데이터 로드
# ------------------------------------------------------
ann_df = pd.read_csv(annotations_csv)
map_df = pd.read_csv(mapping_csv)

# 문자열 정리
ann_df["dl_idx"] = ann_df["dl_idx"].astype(str).str.strip()
map_df["categories_id"] = map_df["categories_id"].astype(str).str.strip()

# ------------------------------------------------------
# 4️⃣ dl_idx → class_id 매핑
# ------------------------------------------------------
merged = ann_df.merge(
    map_df[["categories_id", "class_id"]],
    left_on="dl_idx",
    right_on="categories_id",
    how="inner"
)

# ------------------------------------------------------
# 5️⃣ bbox 문자열 파싱 → YOLO 정규화 좌표 변환
# ------------------------------------------------------
def bbox_to_yolo(bbox, img_w, img_h):
    """
    COCO 형식 [x, y, w, h] → YOLO 정규화 좌표로 변환
    """
    try:
        # bbox 문자열 "[100.5, 200.2, 50.0, 30.0]" → 숫자 리스트
        if isinstance(bbox, str):
            bbox = bbox.strip("[]").split(",")
        bbox = [float(v) for v in bbox]
        if len(bbox) != 4:
            return None
        x, y, w, h = bbox
    except Exception:
        return None

    # 정규화 변환
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    w_norm = w / img_w
    h_norm = h / img_h

    return x_center, y_center, w_norm, h_norm

# ------------------------------------------------------
# 6️⃣ YOLO txt 파일 생성
# ------------------------------------------------------
for file_name, group in merged.groupby("file_name"):
    label_lines = []

    for _, row in group.iterrows():
        bbox = row["bbox"]
        result = bbox_to_yolo(bbox, row["width"], row["height"])
        if result is None:
            continue

        x_center, y_center, w_norm, h_norm = result
        class_id = int(row["class_id"])
        label_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}")

    # 파일 저장
    label_path = os.path.join(output_dir, os.path.splitext(file_name)[0] + ".txt")
    with open(label_path, "w", encoding="utf-8") as f:
        f.write("\n".join(label_lines))

print(f"✅ YOLO 포맷 라벨 파일 생성 완료 → {output_dir}")
print(f"총 생성된 라벨 파일 수: {len(merged['file_name'].unique())}")

✅ YOLO 포맷 라벨 파일 생성 완료 → /Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/data/yolo_label_integration
총 생성된 라벨 파일 수: 2265


In [9]:
import os
import pandas as pd

# ------------------------------------------------------
# 1️⃣ 경로 설정
# ------------------------------------------------------
base_dir = "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/data/yolo_label_integration"

# ------------------------------------------------------
# 2️⃣ txt 파일 탐색 및 class_id 추출
# ------------------------------------------------------
records = []

for fname in os.listdir(base_dir):
    if not fname.endswith(".txt"):
        continue

    fpath = os.path.join(base_dir, fname)
    base_name = os.path.splitext(fname)[0]  # 확장자 제외 파일명

    try:
        with open(fpath, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                parts = line.split()
                class_id = parts[0]  # 첫 번째 컬럼이 class_id
                records.append({
                    "file_name": base_name,
                    "class_id": int(class_id)
                })
    except Exception as e:
        print(f"[WARN] {fname}: {e}")

# ------------------------------------------------------
# 3️⃣ DataFrame 생성
# ------------------------------------------------------
df = pd.DataFrame(records)

# ------------------------------------------------------
# 4️⃣ 결과 출력 및 저장
# ------------------------------------------------------
print(f"✅ 총 {len(df)}개 라벨 항목 추출 완료.")
print(df.head(10))

# CSV로 저장
output_path = os.path.join(base_dir, "yolo_label_integration_summary.csv")
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"💾 CSV 저장 완료 → {output_path}")

✅ 총 8137개 라벨 항목 추출 완료.
                                          file_name  class_id
0  K-003743-005886-012778-038954_0_2_0_2_90_000_200        10
1  K-003743-005886-012778-038954_0_2_0_2_90_000_200        17
2  K-003483-016232-027777-034597_0_2_0_2_90_000_200        50
3  K-003483-016232-027777-034597_0_2_0_2_90_000_200        67
4  K-003483-016232-027777-034597_0_2_0_2_90_000_200        20
5  K-003483-016232-027777-034597_0_2_0_2_90_000_200         3
6  K-003483-020877-025367-036637_0_2_0_2_70_000_200        69
7  K-003483-020877-025367-036637_0_2_0_2_70_000_200        34
8  K-003483-020877-025367-036637_0_2_0_2_70_000_200        45
9  K-003483-020877-025367-036637_0_2_0_2_70_000_200         3
💾 CSV 저장 완료 → /Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/data/yolo_label_integration/yolo_label_integration_summary.csv


In [16]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold

# ------------------------------------------------------
# 1️⃣ 경로 설정
# ------------------------------------------------------
src_dir = "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/data/yolo_label_integration"
dst_root = "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/dataset/2_yolo/labels"

# ------------------------------------------------------
# 2️⃣ CSV 로드 (이전 단계에서 만든 summary)
# ------------------------------------------------------
csv_path = os.path.join(src_dir, "yolo_label_integration_summary.csv")
df = pd.read_csv(csv_path)

# 필수 컬럼 확인
assert {"file_name", "class_id"}.issubset(df.columns), "file_name / class_id 컬럼이 필요합니다."

# ------------------------------------------------------
# 3️⃣ StratifiedGroupKFold 사용
#     - class_id 비율 유지
#     - file_name 단위로 분할 (같은 이미지가 다른 split에 안 들어가도록)
# ------------------------------------------------------
df["file_name"] = df["file_name"].astype(str)

# 그룹 단위 분할을 위한 unique file list 생성
groups = df["file_name"]
y = df["class_id"]

sgkf = StratifiedGroupKFold(n_splits=10, shuffle=True, random_state=42)

splits = list(sgkf.split(df, y, groups))
# 8:1:1 비율 -> 첫 8, 다음 1, 마지막 1 split 사용
train_idx, temp_idx = splits[0]  # 첫 번째 10개 fold 중 하나를 train
val_idx, test_idx = splits[1]    # 다음 2개 fold로 val, test

# train, val, test DataFrame 분리
train_df = df.iloc[train_idx]
val_df = df.iloc[val_idx]
test_df = df.iloc[test_idx]

print(f"✅ train: {len(train_df)}, val: {len(val_df)}, test: {len(test_df)}")

# ------------------------------------------------------
# 4️⃣ 폴더 구조 생성
# ------------------------------------------------------
for sub in ["train", "val", "test"]:
    os.makedirs(os.path.join(dst_root, sub), exist_ok=True)

# ------------------------------------------------------
# 5️⃣ 파일 복사 함수 정의
# ------------------------------------------------------
def copy_files(file_list, subset_name):
    dst_dir = os.path.join(dst_root, subset_name)
    copied = 0
    for fname in file_list:
        src_path = os.path.join(src_dir, f"{fname}.txt")
        dst_path = os.path.join(dst_dir, f"{fname}.txt")
        if os.path.exists(src_path):
            shutil.copy(src_path, dst_path)
            copied += 1
    print(f"📦 {subset_name} 세트: {copied}개 파일 복사 완료 → {dst_dir}")

# ------------------------------------------------------
# 6️⃣ 실제 복사 수행
# ------------------------------------------------------
copy_files(train_df["file_name"].unique(), "train")
copy_files(val_df["file_name"].unique(), "val")
copy_files(test_df["file_name"].unique(), "test")

print("\n🎯 YOLO 라벨 분할 및 복사 완료!")

✅ train: 7306, val: 7331, test: 806
📦 train 세트: 3380개 파일 복사 완료 → /Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/dataset/2_yolo/labels/train
📦 val 세트: 3380개 파일 복사 완료 → /Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/dataset/2_yolo/labels/val
📦 test 세트: 374개 파일 복사 완료 → /Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/dataset/2_yolo/labels/test

🎯 YOLO 라벨 분할 및 복사 완료!


In [14]:
import os
import shutil

# ------------------------------------------------------
# 1️⃣ 경로 설정
# ------------------------------------------------------
label_root = "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/dataset/2_yolo/labels"
src_image_dirs = [
    "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/data/add/images",
    "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/dataset/1_project/train_images",
]
dst_image_root = "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/dataset/2_yolo/images"
log_path = "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/data/missing_images.log"

# ------------------------------------------------------
# 2️⃣ 로그파일 초기화
# ------------------------------------------------------
with open(log_path, "w", encoding="utf-8") as logf:
    logf.write("🚫 누락된 이미지 및 삭제된 라벨 목록\n\n")

# ------------------------------------------------------
# 3️⃣ 분할 폴더 순회 (train, val, test)
# ------------------------------------------------------
for subset in ["train", "val", "test"]:
    label_dir = os.path.join(label_root, subset)
    dst_dir = os.path.join(dst_image_root, subset)
    os.makedirs(dst_dir, exist_ok=True)

    txt_files = [f for f in os.listdir(label_dir) if f.endswith(".txt")]
    missing_count = 0
    copied_count = 0
    deleted_labels = 0

    for txt in txt_files:
        base_name = os.path.splitext(txt)[0]
        dst_path = os.path.join(dst_dir, f"{base_name}.png")
        txt_path = os.path.join(label_dir, txt)

        # 두 개의 소스 디렉토리 중 하나라도 있으면 복사
        found = False
        for src_dir in src_image_dirs:
            src_path = os.path.join(src_dir, f"{base_name}.png")
            if os.path.exists(src_path):
                shutil.copy(src_path, dst_path)
                copied_count += 1
                found = True
                break

        if not found:
            missing_count += 1
            # 라벨(txt) 삭제
            if os.path.exists(txt_path):
                os.remove(txt_path)
                deleted_labels += 1
            with open(log_path, "a", encoding="utf-8") as logf:
                logf.write(f"[{subset}] {base_name}.png 없음 → 라벨 삭제됨: {txt_path}\n")

    print(f"📦 {subset.upper()} 세트 → 복사: {copied_count}개 / 누락: {missing_count}개 / 라벨삭제: {deleted_labels}개")

print("\n✅ 이미지 복사 및 정리 완료!")
print(f"🪵 로그 파일 저장 위치: {log_path}")

📦 TRAIN 세트 → 복사: 3371개 / 누락: 0개 / 라벨삭제: 0개
📦 VAL 세트 → 복사: 3375개 / 누락: 0개 / 라벨삭제: 0개
📦 TEST 세트 → 복사: 370개 / 누락: 0개 / 라벨삭제: 0개

✅ 이미지 복사 및 정리 완료!
🪵 로그 파일 저장 위치: /Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/data/missing_images.log


In [15]:
#이미지 박스 정규화
import os
from PIL import Image

# -------------------------------------------------------
# 1️⃣ 경로 설정
# -------------------------------------------------------
# YOLO 라벨 파일 폴더 경로
label_root = "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/dataset/2_yolo/labels"

# 이미지 파일이 저장된 경로들 (원본 또는 add 폴더)
image_roots = [
    "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/data/add/images",
    "/Users/apple/Documents/ai_engineer/Tablet-Detection-with-Object-Detection/dataset/1_project/train_images"
]

# -------------------------------------------------------
# 2️⃣ 보조 함수 — 이미지 크기 탐색
# -------------------------------------------------------
def find_image_size(base_name):
    """라벨 이름(base_name)에 해당하는 이미지의 (width, height)를 반환"""
    for root in image_roots:
        for ext in [".png", ".jpg", ".jpeg"]:
            img_path = os.path.join(root, f"{base_name}{ext}")
            if os.path.exists(img_path):
                with Image.open(img_path) as img:
                    return img.size  # (width, height)
    return None

# -------------------------------------------------------
# 3️⃣ 정규화 함수
# -------------------------------------------------------
def normalize_bbox(xc, yc, w, h, img_w, img_h):
    """픽셀 단위를 0~1 범위로 정규화"""
    return xc / img_w, yc / img_h, w / img_w, h / img_h

# -------------------------------------------------------
# 4️⃣ 라벨 파일 순회 및 변환
# -------------------------------------------------------
count_total = 0
count_converted = 0
missing_images = []

for subset in ["train", "val", "test"]:
    label_dir = os.path.join(label_root, subset)
    if not os.path.exists(label_dir):
        continue

    print(f"📂 변환 중: {subset}")
    for txt_file in os.listdir(label_dir):
        if not txt_file.endswith(".txt"):
            continue

        base_name = os.path.splitext(txt_file)[0]
        label_path = os.path.join(label_dir, txt_file)
        img_size = find_image_size(base_name)
        count_total += 1

        if img_size is None:
            missing_images.append(base_name)
            continue

        img_w, img_h = img_size
        new_lines = []

        with open(label_path, "r", encoding="utf-8") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue

                cls_id, xc, yc, w, h = parts
                # float 변환
                xc, yc, w, h = map(float, [xc, yc, w, h])

                # 값이 1을 초과하는 경우 정규화 필요
                if xc > 1 or yc > 1 or w > 1 or h > 1:
                    xc, yc, w, h = normalize_bbox(xc, yc, w, h, img_w, img_h)
                    count_converted += 1

                new_lines.append(f"{cls_id} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}")

        # 변환 결과를 같은 파일에 overwrite
        with open(label_path, "w", encoding="utf-8") as f:
            f.write("\n".join(new_lines) + "\n")

print("\n✅ 변환 완료!")
print(f"총 라벨 파일 수: {count_total}")
print(f"정규화 수행된 파일 수: {count_converted}")
if missing_images:
    print(f"⚠️ 이미지가 없는 라벨 파일: {len(missing_images)}개")
    print("\n".join(missing_images[:10]), "...")

📂 변환 중: train
📂 변환 중: val
📂 변환 중: test

✅ 변환 완료!
총 라벨 파일 수: 7116
정규화 수행된 파일 수: 12
